## Code to get disease names from UMLS

In [1]:
import requests
import xml.etree.ElementTree as ET
from xml.etree import ElementTree
from tqdm import tqdm
ticket = 'd57d8be8-295c-4428-9b47-8882eaf9ed75'

In [2]:
import pandas as pd

In [3]:
diseases = pd.read_csv('../data/SND/disease_snf14.csv', index_col=0)

In [4]:
disease_names = list(diseases.index)

In [5]:
def get_mesh_id(concept_id, ticket):
    try:
        apikey = ticket
        version = 'current'
        string = concept_id
        uri = "https://uts-ws.nlm.nih.gov"
        content_endpoint = "/rest/content/"+version+"/CUI/"+concept_id + "/atoms"
        full_url = uri+content_endpoint

        query = {'string':string,'apiKey':apikey}

        r = requests.get(full_url,params=query)
        # r.raise_for_status()
        r.encoding = 'utf-8'
        outputs  = r.json()
        try:
            # print("here")
            items = (([outputs['result']])[0])
            # print(items)
            # print("out")
            return items
        except:
            # print(outputs)
            return 'NONE' 
    except:
        return 'NONE'

In [ ]:
ids = {}
k = 0
for disease in tqdm(disease_names):
    mesh_id_all = get_mesh_id(disease, ticket)
    mesh_id = []
    try:
        for i in range(0,len(mesh_id_all)):
            # print(mesh_id_all[i]['sourceDescriptor'].split('/')[-1])
            try:
                mesh_id.append(mesh_id_all[i]['sourceDescriptor'].split('/')[-1])
            except:
                mesh_id.append('NONE')
    except:
        mesh_id = ["NONE"]
    k+=1
    # if k == 10:
    #     break
    ids[disease] = mesh_id

  3%|█▌                                                   | 23/803 [00:35<18:34,  1.43s/it]

In [ ]:
len(disease_names)

In [ ]:
len(ids.keys())

In [ ]:
ids

In [ ]:
import json

with open("SND_diseases.json", "w") as f:
    # Write the dictionary to the file in JSON format
    json.dump(ids, f)

In [ ]:
import json

# Open the JSON file and read its contents
with open('../../biosnap/gene_dict_disease.json', 'r') as f:
    data = f.read()

# Parse the JSON data into a Python dictionary
json_dict = json.loads(data)

In [ ]:
keys = set(json_dict.keys())

In [ ]:
count = 0 
correct_names = {} 
for diseases in ids.keys():
    lst = ids[diseases]
    common = set(lst).intersection(set(keys))
    if common != set():
        count+=1
        correct_names[diseases] = list(common)[0]
print(count)

In [ ]:
correct_names

In [ ]:
import json

with open("../data/SND/SNF_diseases_matched.json", "w") as f:
    # Write the dictionary to the file in JSON format
    json.dump(correct_names, f)